<a href="https://colab.research.google.com/github/MiniRedTrout/NeuroLess/blob/main/2)Hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
from torch.nn import functional as F
import torch.optim as optim
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from torch.optim.lr_scheduler import ReduceLROnPlateau

Ввели библиотеки)

In [2]:
torch.cuda.is_available()

True

In [3]:
np.random.seed(42)
iris =  load_iris()
X = iris.data
y = iris.target

In [4]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

Стандартизируем, помогло, чтобы при обучении ошибка не скакала

In [5]:
scal = StandardScaler()
X_train = scal.fit_transform(X_train)
X_test = scal.transform(X_test)
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

In [6]:
train = TensorDataset(X_train,y_train)
test = TensorDataset(X_test,y_test)
train = DataLoader(train,batch_size=16)
test = DataLoader(test,batch_size=16)


Моделька

In [7]:
class iris_model(nn.Module):
    def __init__(self, input_size, hidden_size):
      super(iris_model,self).__init__()
      self.layers = nn.Sequential(
          nn.Linear(input_size, hidden_size),
          nn.BatchNorm1d(hidden_size),
          nn.ReLU(),
          nn.Dropout(0.4),
          nn.Linear(hidden_size, hidden_size // 4),
          nn.BatchNorm1d(hidden_size // 4),
          nn.ReLU(),
          nn.Dropout(0.4),
          nn.Linear(hidden_size // 4, 3)
      )
    def forward(self, x):
      return self.layers(x)

In [8]:
device = torch.device('cuda')
model = iris_model(input_size=4,hidden_size=64)

In [9]:
model.to(device)

iris_model(
  (layers): Sequential(
    (0): Linear(in_features=4, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=64, out_features=16, bias=True)
    (5): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.4, inplace=False)
    (8): Linear(in_features=16, out_features=3, bias=True)
  )
)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

Тренирую

In [11]:
def train_epoch(model, dtload, criterion, optimizer, device):
    for epoch in range(1000):
      model.train()
      losses = []
      for batchX, batchy in dtload:
          batchX, batchy = batchX.to(device), batchy.to(device)
          optimizer.zero_grad()
          out = model(batchX)
          loss = criterion(out, batchy)
          loss.backward()
          optimizer.step()
          losses.append(loss.item())
      if epoch % 100 == 0:
          print(f'Epoch {epoch}  loss {sum(losses)/len(dtload)}')

train_epoch(model,train,criterion,optimizer,device)

Epoch 0  loss 0.9044895395636559
Epoch 100  loss 0.07253990636672825
Epoch 200  loss 0.09083488397300243
Epoch 300  loss 0.12992292328272015
Epoch 400  loss 0.08159535811864771
Epoch 500  loss 0.07232164032757282
Epoch 600  loss 0.02043365640565753
Epoch 700  loss 0.060011376393958926
Epoch 800  loss 0.054398815729655325
Epoch 900  loss 0.05681553762406111


Оценка

In [12]:
def val(model, dtload, device):
    model.eval()
    all_predictions = []

    with torch.no_grad():
        for batch in dtload:
            batchX, batchy = batch
            batchX, batchy = batchX.to(device), batchy.to(device)

            out = model(batchX)
            predictions = torch.argmax(out, dim=1)

            all_predictions.extend(predictions.cpu().numpy())

    return np.array(all_predictions)

from sklearn.metrics import classification_report

predictions = val(model, test, device)
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, predictions))

Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

